In [1]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import requests
import json
import os

from openai import OpenAI

In [11]:
client = OpenAI()

In [43]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "developer",
            "content": [{
                "type": "text",
                "text": """
                    Users are playing an RPG. They will ask questions pertaining to the rules, setting, lore,
                    and current situation. You will accept the query and return a list of relevant keywords that
                    can be used to locate relevant sections of the source material.

                    You will NOT attempt to answer the questions yourself. Your only role is to extract useful keywords
                    to be used in a search. Organize them in order of likely relevance, from most relevant to least.

                    Return the keywords in a comma delimited format with no quote characters or extraneous punctuation.
                """
            }],
        },
        {
            "role": "user",
            "content": [{
                "type": "text",
                "text": "My Strength is 16. I am wearing full plate armor. Does this impact my Swim speed?"
            }]
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Strength, Swim speed, Full plate armor, Movement penalties, Armor effects, Swimming rules', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [44]:
completion.choices[0].message.content

'Strength, Swim speed, Full plate armor, Movement penalties, Armor effects, Swimming rules'

In [45]:
def elastic_request(data=None, method=None, url=None):
    if method == None:
        method = requests.get
    if data:
        data = json.dumps(data)    

    return method(f"https://192.168.1.153:9200/{url}",
                  headers={
                    "Content-Type": "application/json",
                    "Accept": "application/json",
                    "Authorization": f"ApiKey {os.getenv('ELASTIC_API_KEY')}"
                  },
                  verify=False,
                  data=data)

In [46]:
response = elastic_request(url="players-handbook-embedded/_search",
                           data={
                                "query": {
                                    "match": {
                                        "attachment.content": {
                                            "query": completion.choices[0].message.content,
                                            "operator": "or",
                                            "boost": 0.7
                                        }
                                    }
                                },
                                "knn": {
                                    "field": "content-embedding",
                                    "k": 10,
                                    "boost": 0.3,
                                    "num_candidates": 10,
                                    "query_vector_builder": {
                                        "text_embedding": { 
                                            "model_id": "open-ai-embeddings", 
                                            "model_text": "My Strength is 16. I am wearing full plate armor. Does this impact my Swim speed?" 
                                        }
                                    }
                                }
                           })

/home/vscode/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.153'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [47]:
hits = response.json()["hits"]["hits"]

In [48]:
[h["_source"]["attachment"]["content"] for h in hits]

['CH. 7: USING ABILITY SCORES CH. 9: COMBAT\n\nChapter 8: Adventuring\n\nDelving into the ancient Tomb of Horrors, slipping through the back alleys of Waterdeep, hacking a fresh trail through the thick jungles on the Isle of Dread\n\n—these are the things that Dungeons & Dragons adventures are made of. Your character in the game might explore forgotten ruins and uncharted lands,\n\nuncover dark secrets and sinister plots, and slay foul monsters. And if all goes well, your character will survive to claim rich rewards before embarking on\n\na new adventure.\n\nThis chapter covers the basics of the adventuring life, from the mechanics of movement to the complexities of social interaction. The rules for resting are\n\nalso in this chapter, along with a discussion of the activities your character might pursue between adventures.\n\nWhether adventurers are exploring a dusty dungeon or the complex relationships of a royal court, the game follows a natural rhythm, as outlined in the\n\nbook’s 

In [49]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "developer",
            "content": [{
                "type": "text",
                "text": """
                    Users are playing an RPG. They will ask questions pertaining to the rules, setting, lore,
                    and current situation. You will answer the question based on the content provided.
                    
                    Use only the content provided. Do not base your answers on anything other than the content.
                    Quote the content verbatim if possible (but only if it is relevant to the question!).
                    Otherwise, keep your answer as brief as possible. 
                """
            }],
        },
        {
            "role": "assistant",
            "content": [{ "type": "text", "text": h["_source"]["attachment"]["content"]} for h in hits[0:2]]
        },
        {
            "role": "user",
            "content": [{
                "type": "text",
                "text": "My Strength is 16. I am wearing full plate armor. Does this impact my Swim speed?"
            }]
        }
    ]
)

In [50]:
completion.choices[0].message.content

'Yes, wearing full plate armor impacts your swim speed. When you swim, each foot of movement costs 1 extra foot (2 extra feet in difficult terrain) if you are not proficient with swimming in armor. Since you are in full plate armor and it typically imposes a disadvantage on Dexterity (Stealth) checks, it would also hinder your ability to swim efficiently. You would have to manage your movement accordingly.'

In [51]:
benchmark_questions = [
    "Can I cast Hunter's Mark on an Invisible enemy?",
    "My Strength is 18. I will take a running jump. How far can I make it?",
    "My Strength is 18. I am wearing full plate armor. How does this impact my ability to make a running jump?",
    "My Strength is 16. I am wearing full plate armor. Does this impact my Swim speed?"
]